In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

cancer = load_breast_cancer()

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaled = scaler.fit_transform(cancer.data)

X_train, X_test, y_train, y_test = train_test_split(scaled, cancer.target, test_size=0.3, random_state=0)

In [3]:
from sklearn.metrics import accuracy_score, roc_auc_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_clf_preds = lr_clf.predict(X_test)

print('accuracy: {0:.3f}'.format(accuracy_score(y_test,lr_clf_preds)))
print('roc_auc: {0:.3f}'.format(roc_auc_score(y_test,lr_clf_preds)))

accuracy: 0.977
roc_auc: 0.972


In [8]:
from sklearn.model_selection import GridSearchCV

params = {'penalty':['l2','l1'], # 규제의 유형 l2 or l1. 디폴트는 l2
          'C':[0.01, 0,1, 1, 5, 10]} # 규제의 강도를 조절하는 alpha값의 역수. C값이 작을수록 규제 강도가 크다.

grid_clf = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=3)
grid_clf.fit(scaled, cancer.target)
print('최적 하이퍼 파라미터: {0}, 최적 평균 정확도: {1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this trai

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


최적 하이퍼 파라미터: {'C': 1, 'penalty': 'l2'}, 최적 평균 정확도: 0.975


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


### 이진분류가 아닌 경우 적용방법에 대한 저자의 답변
Logistic Regression은 기본적으로 이진 분류(Binary classification) 인데, 이를 multi classification에 적용하는 방법은 주로 One Vs RestClassifier 방법입니다.One Vs RestClassifier은 이진 분류가 아닌 타겟값이 여러개인 multi classification(타겟값의 유형이 여러개)에서 일반적으로 언급되는 개념입니다. 

One Vs RestClassifier은 가령 100개의 데이터 중에 타겟값 유형이 사과, 바나나, 배, 딸기 4개 유형이 있다고 할 때, 4개의 이진 classifier를 만드는데 개별 이진 classifer는 각 class 값/class값 아님으로 분리하여 학습하는 방식입니다.

예를 들어 사과 이진 classifer는 결과값을 사과/사과아님 으로 간주하고 여기에 해당하는 데이터 세트를 모아서 학습합니다.  마찬가지로 바나나 이진 classifier는 결과값을 바나나/바나나 아님과 같은 형태로 만들어서 여기에 해당하는 데이터 세트를 모아서 학습합니다. 즉  자기가 담당하는 클래스값 외에는 나머지(Rest)는 모두 negative로 처리하는 방식입니다.

이후에 4개의 classifier중 가장 높은 확률로 예측하는 클래스를 선택하여 multi classification을 구현하는 방식입니다.

사이킷런의 LogisticRegression 객체는 생성자로 multi_class 파라미터를 가집니다. multi_class='ovr' 일경우 binary classification, 'multinomial' 일 경우 multi classification, 'auto' 일 경우 타겟값 유형을 자동으로 해석하여 binary 또는 multinomial을 적용합니다.

clf = LogisticRegression(multi_class='auto')

디폴트가 auto 이므로 LogisticRegression() 으로 하면 타겟값의 유형에 따라 자동으로 binary 또는 multi classification 을 적용 합니다.